In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

/home/yuncong/Brain/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages

Setting environment for Precision WorkStation
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif


No vtk
No vtkNot using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD595/MD595_prep2_thumbnail/MD595-IHC12-2015.09.15-01.21.39_MD595_2_0035_prep2_thumbnail.tif
/medi

Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


/media/yuncong/BstemAtlasData/CSHL_data_processed/MD658/MD658_prep2_thumbnail/MD658-N18-2017.03.31-17.34.22_MD658_2_0053_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD661/MD661_prep2_thumbnail/MD662&661-F30-2017.06.05-14.49.48_MD661_1_0088_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif
/media/yuncong/BstemAtlasData/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail_Ntb/ChatCryoJane201710_slide1-S1_prep2_thumbnail_Ntb.tif


In [2]:
from sklearn.externals import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier 

sys.path.append('/home/yuncong/csd395/xgboost/python-package')
try:
    from xgboost.sklearn import XGBClassifier
except:
    sys.stderr.write('xgboost is not loaded.')

In [3]:
batch_size = 256
model_dir_name = 'inception-bn-blue'
model_name = 'inception-bn-blue'
model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
                                   num_gpus=1, batch_size=batch_size)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [7]:
train_stacks = ['MD589', 'MD585']
test_stacks = ['MD594']

In [8]:
stack_stain = {'MD585': 'N', 'MD589': 'N', 'MD594': 'N'}

# Number of sections on which to sample examples from.
stack_section_number = defaultdict(dict)

for name_u in all_known_structures:
    for st in train_stacks:
        stack_section_number[st][name_u] = 99
#         if name_u == '4N' or name_u == '10N':
#             train_stack_section_number[st][name_u] = 20
#         else:
#             train_stack_section_number[st][name_u] = 10
#         train_stack_section_number[st][name_u] = 10
    for st in test_stacks:
        stack_section_number[st][name_u] = 99

stack_section_number.default_factory = None

In [9]:
win_id = 7

In [10]:
grid_indices_lookup_allStacks = {}

for stack in train_stacks + test_stacks:
    try:
#         grid_indices_lookup_allStacks[stack] = \
#         DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
#                                                            by_human=False, timestamp='latest',
#                                                            detector_id_f=1,
#                                                           return_locations=True)            

        grid_indices_lookup_allStacks[stack] = \
        DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
                                                           by_human=True, timestamp='latest',
                                                          return_locations=True)            

    except Exception as e:
        print e
        sys.stderr.write("Fail to load annotation grid lookup for %s.\n" % stack)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/home/yuncong/CSHL_labelings_v3/MD589" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  10042017100807
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD585" "/home/yuncong/CSHL_labelings_v3/MD585" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  08012017212649
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD594" "/home/yuncong/CSHL_labelings_v3/MD594" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  07302017183604


In [11]:
from itertools import chain
all_labels = sorted(list(set(chain.from_iterable(set(grid_indices_lookup_allStacks[st].columns.tolist()) 
                                                 for st in train_stacks + test_stacks))))

In [18]:
def sample_addresses(stacks, structure):
        
    positive_labels = [structure]
    negative_labels = get_negative_labels(structure, 'neg_has_all_surround', 
                                          margin_um=500, labels_found=all_labels)

    positive_addresses_all_stacks = {}
    negative_addresses_all_stacks = {}
        
    for stack in stacks:

        candidate_sections = list(chain(*[grid_indices_lookup_allStacks[stack][pl].dropna(how='any').index.tolist() 
                                      for pl in positive_labels]))

        n_sections = stack_section_number[stack][structure]

        if stack_stain[stack] == 'F':
            neurotrace_sections = []
            nissl_sections = []
            for sec in candidate_sections:
                if metadata_cache['sections_to_filenames'][stack][sec].split('-')[1][0] == 'F':
                    neurotrace_sections.append(sec)
                else:
                    nissl_sections.append(sec)
            sampled_sections = np.random.choice(neurotrace_sections, min(len(neurotrace_sections), n_sections), replace=False)
        else:
            sampled_sections = np.random.choice(candidate_sections, min(len(candidate_sections), n_sections), replace=False)

        positive_addresses_all_stacks[stack] = sorted([(stack, sec, tuple(loc))
for nl in set(positive_labels) & set(grid_indices_lookup_allStacks[stack].columns)
  for sec, locs in grid_indices_lookup_allStacks[stack][nl].loc[sampled_sections].dropna().iteritems()
  for loc in locs])

        negative_addresses_all_stacks[stack] = sorted([(stack, sec, tuple(loc))
for nl in set(negative_labels) & set(grid_indices_lookup_allStacks[stack].columns)
  for sec, locs in grid_indices_lookup_allStacks[stack][nl].loc[sampled_sections].dropna().iteritems()
  for loc in locs])

    positive_addresses = sum(positive_addresses_all_stacks.values(), [])
    negative_addresses = sum(negative_addresses_all_stacks.values(), [])

    del positive_addresses_all_stacks, negative_addresses_all_stacks

    return positive_addresses, negative_addresses

In [19]:
# train_scheme = 'stretch_min_max'
# train_scheme = 'normalize_mu_sigma_global_(-1,5)'
train_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,5)'
# train_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,9)'
# train_scheme = 'none'

# test_scheme = 'stretch_min_max'
# test_scheme = 'normalize_mu_sigma_global_(-1,5)'
test_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,5)'
# test_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,9)'
# test_scheme = 'none'

In [20]:
# existing_classifier_id = 998 # If set, not train from scratch
existing_classifier_id = None

In [21]:
# classifier_id = 998

In [22]:
# features_dict = {(scheme, tfv): {} for scheme in schemes for tfv in transforms}
features_dict = defaultdict(dict)

In [23]:
# for structure in all_known_structures:
for structure in ['7N']:

    ############## Sample and Load training feature vectors #########################################

    positive_addresses_traindata, negative_addresses_traindata = \
    sample_addresses(train_stacks, structure)
    
    print '# positive train =', len(positive_addresses_traindata)
    print '# negative train =', len(negative_addresses_traindata)
    
    addresses_to_compute = positive_addresses_traindata + negative_addresses_traindata

    for variant in [0]:
        features_loaded = read_cnn_features(addresses=addresses_to_compute, 
                                            scheme=train_scheme, win_id=7, prep_id=2, 
                                            model=model, mean_img=mean_img, model_name=model_name, 
                                            batch_size=batch_size)

        for addr, f in izip(addresses_to_compute, features_loaded):
            features_dict[(train_scheme, variant)][addr] = f

        del features_loaded

    ############## Sample and Load test feature vectors #############################################
    
    positive_addresses_testdata, negative_addresses_testdata = \
    sample_addresses(test_stacks, structure)
    
    print '# positive test =', len(positive_addresses_testdata)
    print '# negative test =', len(negative_addresses_testdata)
    
    addresses_to_compute = positive_addresses_testdata + negative_addresses_testdata

    for variant in [0]:
        features_loaded = read_cnn_features(addresses=addresses_to_compute, 
                                            scheme=train_scheme, win_id=7, prep_id=2, 
                                            model=model, mean_img=mean_img, model_name=model_name, 
                                            batch_size=batch_size)

        for addr, f in izip(addresses_to_compute, features_loaded):
            features_dict[(test_scheme, variant)][addr] = f

        del features_loaded
            
    ########################################################################################
    
    # n_train_list = [10, 100, 200, 500, 1000, 2000, 5000, 10000, 15000]
#     n_train_list = [10, 1000]
#     n_train_list = [1000, 5000, 15000]
    n_train_list = [15000]
    test_metrics_all_ntrain = defaultdict(lambda: defaultdict(list))
    train_metrics_all_ntrain = defaultdict(lambda: defaultdict(list))

    for n_train in n_train_list:

        for trial in range(3):
            
            ##### Sample from training pool the required number of examples ######

            # If train and test data are from different sets
            n_train_pos = min(n_train, len(positive_addresses_traindata))
#             if len(positive_addresses_traindata) < n_train_pos:
#                 continue
            training_pos_indices = np.random.choice(range(len(positive_addresses_traindata)), n_train_pos, replace=False)
            
            n_test_pos = min(len(positive_addresses_testdata), 1000)
            test_pos_indices = np.random.choice(range(len(positive_addresses_testdata)), n_test_pos, replace=False)

            # If train and test are from same set
        #     n_pos_total = len(positive_addresses)
        #     n_train_pos = 1000
        #     training_pos_indices = np.random.choice(range(n_pos_total), n_train_pos, replace=False)
        #     test_pos_indices = np.random.choice(np.setdiff1d(range(n_pos_total), training_pos_indices),
        #                                         size=min(2000, n_pos_total-n_train_pos), replace=False)
        #     n_test_pos = len(test_pos_indices)

            # If train and test data are from different sets
            n_train_neg = n_train_pos
            training_neg_indices = np.random.choice(range(len(negative_addresses_traindata)), n_train_neg, replace=False)
            
            n_test_neg = min(len(negative_addresses_testdata), 1000)
            test_neg_indices = np.random.choice(range(len(negative_addresses_testdata)), n_test_neg, replace=False)

            # If train and test are from same set
        #     n_neg_total = len(negative_addresses)
        #     n_train_neg = 1000
        #     training_neg_indices = np.random.choice(range(n_neg_total), n_train_neg, replace=False)
        #     test_neg_indices = np.random.choice(np.setdiff1d(range(n_neg_total), training_neg_indices), 
        #                                         size=min(2000, n_pos_total-n_train_pos), replace=False)
        #     n_test_neg = len(test_neg_indices)

            print "Training: %d positive, %d negative" % (n_train_pos, n_train_neg)
            print "Test: %d positive, %d negative" % (n_test_pos, n_test_neg)

            ################

            # If train and test data are from different sets
            addresses_train_pos = [positive_addresses_traindata[i] for i in training_pos_indices]
            addresses_train_neg = [negative_addresses_traindata[i] for i in training_neg_indices]

            #################

#             for augment_training in [True, False]:
            for augment_training in [False]:
        
                
                feature_classifier_alg = 'lr'
        #             feature_classifier_alg = 'xgb2'
        #             feature_classifier_alg = 'lin_svc'
        #             feature_classifier_alg = 'lin_svc_calib'
                sample_weights = None   

                if augment_training:
                    train_transforms = range(8)
                else:
                    train_transforms = range(1)
                features_train_pos = {(train_scheme, tf_variant): 
                                      [features_dict[(train_scheme, tf_variant)][addr] 
                                       for addr in addresses_train_pos]
                                          for tf_variant in train_transforms}
                features_train_neg = {(train_scheme, tf_variant): 
                                      [features_dict[(train_scheme, tf_variant)][addr] 
                                       for addr in addresses_train_neg]
                                          for tf_variant in train_transforms}

                train_data = np.concatenate([np.r_[features_train_pos[(train_scheme,tf)], 
                                                   features_train_neg[(train_scheme,tf)]] 
                                                for tf in train_transforms])                    
                train_labels = np.concatenate([np.r_[np.ones((len(features_train_pos[(train_scheme,tf)]), )), 
                                                    -np.ones((len(features_train_neg[(train_scheme,tf)]), ))]
                                              for tf in train_transforms])

                if existing_classifier_id is None:
                    clf = train_binary_classifier(train_data, train_labels,
                                       alg=feature_classifier_alg, 
                                       sample_weights=sample_weights)

    #                 del train_data, features_train_pos, features_train_neg

#                     clf_fp = DataManager.get_classifier_filepath(classifier_id=classifier_id, structure=structure)
#                     create_parent_dir_if_not_exists(clf_fp)
#                     joblib.dump(clf, clf_fp)
#                     upload_to_s3(clf_fp)
                else:
                    sys.stderr.write('Load existing classifiers %d\n' % existing_classifier_id)
                    clf = DataManager.load_classifiers(classifier_id=existing_classifier_id)[structure]
                
                ######################### Compute train metrics #########################
                
                train_metrics = compute_classification_metrics(clf.predict_proba(train_data)[:,1], train_labels)
                train_metrics_all_ntrain[n_train][(train_scheme, 'augment' if augment_training else 'no-augment')].append(train_metrics)
                
                ######################### Test ###############################
                
                test_transforms = range(1)
                features_test_pos = {(test_scheme, tf_variant): 
                                      [features_dict[(train_scheme, tf_variant)][addr] 
                                       for addr in addresses_test_pos]
                                          for tf_variant in test_transforms}
                features_test_neg = {(test_scheme, tf_variant): 
                                      [features_dict[(train_scheme, tf_variant)][addr] 
                                       for addr in addresses_test_neg]
                                          for tf_variant in test_transforms}
                
                test_data = np.r_[features_test_pos[(test_scheme,'transform0')], features_test_neg[(test_scheme,'transform0')]]
                test_labels = np.r_[np.ones((len(features_test_pos[(test_scheme,'transform0')]), )), 
                                     -np.ones((len(features_test_neg[(test_scheme,'transform0')]), ))]
                test_metrics = compute_classification_metrics(clf.predict_proba(test_data)[:,1], test_labels)
    #             print "acc@0.5 = %.3f, acc@opt = %.3f, opt_thresh = %.3f, auroc = %.3f, auprc = %.3f" % \
    #             (test_metrics['acc'][0.5], test_metrics['acc'][test_metrics['opt_thresh']], test_metrics['opt_thresh'], test_metrics['auroc'], test_metrics['auprc'])

                test_metrics_all_ntrain[n_train][(test_scheme, 'augment' if augment_training else 'no-augment')].append(test_metrics)

    train_metrics_all_ntrain.default_factory = None
    test_metrics_all_ntrain.default_factory = None
    
    plot_result_wrt_ntrain(extract_one_metric(test_metrics_all_ntrain, 'acc', 0.5), ylabel='Test accuracy@0.5 threshold');
    plot_result_wrt_ntrain(extract_one_metric(test_metrics_all_ntrain, 'auroc'), ylabel='Area under ROC');

    plot_roc_curve(test_metrics_all_ntrain[1000][(test_scheme,
                  'augment')][0]['fp'], 
                   test_metrics_all_ntrain[1000][(test_scheme,
                  'augment')][0]['tp'], 
                  test_metrics_all_ntrain[1000][(test_scheme,
                  'augment')][0]['opt_thresh']);
    
#     import uuid

#     result = {
#         'n_train_sections': train_stack_section_number,
#         'n_test_sections': test_stack_section_number,
#         'train_stain': train_stack_stain,
#         'test_stain': test_stack_stain,
#         'train_stack': train_stack,
#         'test_stack': test_stack,
#         'test_scheme': test_scheme,
#         'train_scheme': train_scheme,
#         'train_metrics_all_ntrain': train_metrics_all_ntrain,
#         'test_metrics_all_ntrain': test_metrics_all_ntrain
#     }

#     create_if_not_exists(ROOT_DIR + '/assessment_results_v2/')
#     save_pickle(result, ROOT_DIR + '/assessment_results_v2/assessment_result_%s.pkl' % str(uuid.uuid1()).split('-')[0])

# positive train = 32694
# negative train = 239050
('MD589', 179)
('MD585', 163)
('MD589', 316)
('MD589', 166)
('MD589', 303)
('MD589', 153)
('MD585', 156)
('MD589', 290)
('MD589', 193)
('MD585', 143)
('MD589', 277)
('MD585', 272)
('MD589', 189)
('MD585', 259)
('MD585', 181)
('MD589', 176)
('MD585', 160)
('MD589', 313)
('MD589', 163)
('MD585', 292)
('MD585', 145)
('MD589', 287)
('MD585', 282)
('MD585', 140)
('MD589', 274)
('MD585', 277)
('MD589', 186)
('MD585', 256)
('MD585', 170)
('MD589', 173)
('MD585', 165)
('MD589', 310)
('MD589', 160)
('MD585', 155)
('MD589', 297)
('MD585', 150)
('MD589', 284)
('MD585', 287)
('MD585', 254)
('MD585', 266)
('MD589', 183)
('MD585', 261)
('MD585', 175)
('MD589', 170)
('MD589', 307)
('MD589', 157)
('MD585', 291)
('MD585', 152)
('MD589', 294)
('MD589', 197)
('MD589', 281)
('MD585', 284)
('MD585', 271)
('MD585', 177)
('MD589', 180)
('MD585', 172)
('MD589', 317)
('MD589', 167)
('MD589', 304)
('MD589', 154)
('MD585', 288)
('MD585', 157)
('MD589', 291)
('MD

Fitting classifier: 16.29 seconds


NameError: global name 'addresses_test_pos' is not defined